<a href="https://colab.research.google.com/github/askani/Project_1/blob/master/RAISE_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
from sentence_transformers import SentenceTransformer, util
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
import pandas as pd

# Download NLTK stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Load SBERT model
model = SentenceTransformer('all-MiniLM-L6-v2')  # Lightweight & effective


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:
# Define category labels and their descriptions
categories = {
    "AI & Technology": "The advancements, integration, and impact of artificial intelligence in computing, software, hardware, and emerging technologies, including cloud computing, edge AI, and IoT",
    "AI & Ethics and Risks": "Ethical considerations, potential biases, fairness, accountability, transparency, and the societal risks posed by artificial intelligence, including misinformation, job displacement, and decision-making biases",
    "AI & Business": "The role of artificial intelligence in optimizing business operations, improving customer service, enhancing decision-making, and driving innovations in industries such as retail, logistics, and supply chain management",
    "AI & Healthcare": "The use of AI in medical diagnostics, drug discovery, patient monitoring, personalized medicine, robotic surgeries, and improving healthcare accessibility",
    "AI & Education": "AI-driven innovations in personalized learning, automated grading, virtual tutoring, education accessibility, and adaptive learning technologies",
    "AI & Finance": "Artificial intelligence in financial services, including algorithmic trading, fraud detection, risk assessment, robo-advisors, and predictive analytics for investment decisions",
    "AI & Society": "The broader impact of AI on social dynamics, human behavior, public policy, and its influence on communities, cultures, and global development",
    "AI & Creativity": "The intersection of artificial intelligence with creative fields such as art, music, literature, fashion, and design, exploring AI-generated content and co-creation with humans",
    "AI & Security": "The role of AI in cybersecurity, threat detection, digital forensics, biometric authentication, and safeguarding systems from cyberattacks and data breaches",
    "AI & Marketing": "The use of AI in marketing, from consumer behavior analysis and personalized recommendations to targeted advertisements, chatbots, and predictive analytics that drive marketing strategies",
    "AI & Environment": "How artificial intelligence is applied in climate science, sustainable energy, wildlife conservation, environmental monitoring, and reducing carbon footprints",
    "AI & Robotics": "The integration of artificial intelligence in robotics for automation, humanoid robotics, industrial robotics, autonomous vehicles, and assistive robots",
    "AI & Law": "The legal implications of AI, including intellectual property rights, AI regulation, liability in AI-driven decisions, and the impact of AI in legal research and case predictions",
    "AI & Politics": "The role of AI in political campaigns, governance, policy-making, voter influence, misinformation detection, and government surveillance technologies",

    "AI & Automation": "The impact of AI-driven automation in various sectors, including workforce transformation, robotic process automation (RPA), and human-machine collaboration",
    "AI & Media & Journalism": "The influence of artificial intelligence in news generation, automated content creation, deepfake detection, fact-checking, and media personalization",

    "AI & Research": "The role of AI in advancing scientific research, including drug discovery, computational science, data-driven insights in fields like biology, physics, and engineering, and accelerating breakthroughs in various domains",

    "AI & Quantum Computing": "The convergence of artificial intelligence with quantum computing, exploring quantum algorithms, AI-driven simulations, and computational advancements in problem-solving",
    "AI & Human Interaction": "The influence of AI on human-computer interaction, natural language processing, conversational AI, affective computing, and social robots"
}

# List of words that need to be included in the cleaned headlines
important_words = ['microsoft', 'google', 'nvidia', 'tesla', 'oracle', 'adm']

# Preprocessing function: Clean text & remove stopwords
def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove special characters
    words = text.lower().split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

# Compute embeddings for category definitions
category_texts = list(categories.values())
category_texts = [preprocess_text(text) for text in category_texts]
category_embeddings = model.encode(category_texts, convert_to_tensor=True)



In [2]:
categories = {
    "AI & Technology": "T1",
    "AI & Technology" : "T2",
    "AI & Education": "E1",
    "AI & Education" : "E2"
}

In [4]:
categories

{'AI & Technology': 'T2', 'AI & Education': 'E2'}

In [4]:
# Function to classify a news article
from sklearn.preprocessing import normalize

def classify_article(article):
    article = preprocess_text(article)
    article_embedding = model.encode(article, convert_to_tensor=True, normalize_embeddings=True)

    # Compute cosine similarity with category embeddings
    similarities = util.cos_sim(article_embedding, category_embeddings)[0].cpu().numpy()

    top_idx = np.argmax(similarities)
    top_category = list(categories.keys())[top_idx]
    top_score = similarities[top_idx]

    # Step 5: Soft thresholding: Assign category unless confidence is too low
    if top_score < 0.2:  # Lowered from 0.5
        return "No relevant category"

    return top_category


# List of News headlines
dataset = pd.read_csv('Dataset_10k.csv')
data_100 = dataset.head(100)
headlines = data_100['title'].tolist()



In [5]:
labels = []

# Classify each headline
for headline in headlines:
    category = classify_article(headline)
    labels.append(category)

data_100['category'] = labels


<ipython-input-5-5085629a1911>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_100['category'] = labels


In [6]:
data_100.category.value_counts()

,count
category,
AI & Law,15
AI & Finance,13
AI & Creativity,10
AI & Business,9
AI & Human Interaction,7
AI & Marketing,7
AI & Politics,6
AI & Healthcare,6
AI & Society,6


In [7]:
pd.set_option('display.max_colwidth', None)
data_100[['title','category']].head(20)

,title,category
0,"will.i.am and Fyilicia on the AI revolution, inclusivity & The Voice UK ...Tech & Science Daily podcast",AI & Media & Journalism
1,Intel Launches World’s First Systems Foundry Designed for the AI Era,AI & Technology
2,The Unique Challenges of Selling Enterprise AI,AI & Law
3,"Contentious California AI bill passes legislature, awaits governor's signature",AI & Politics
4,"Exploring Genius, Creation, and Humanity in the Age of AI",AI & Creativity
5,Conversational AI improves 'fourth trimester' maternal care at Penn Medicine,AI & Human Interaction
6,Disney harnesses AI to drive streaming ad technology,AI & Marketing
7,Will Rinehart: Unpacking the Executive Order on Artificial Intelligence,AI & Business
8,Video: Where Bitcoin and Artificial Intelligence Meet,AI & Finance
9,NVIDIA's relentless rally: AI chip giant eyes new heights​,No relevant category
